In [1]:
import os 
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
import warnings
warnings.filterwarnings("ignore")

os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY_PERSONAL"]
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/opt/homebrew/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style

## Create Tools

In [2]:
search_tool = SerperDevTool()
scrape_website= ScrapeWebsiteTool()

## Creating Agent

In [3]:
data_analyst_agent = Agent(
    role="Data Analyst", 
    goal="Monitor and analyze market data in real-time to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent uses statistical modeling and machine learning \
               to provide crucial insights. With a knack for data, the Data Analyst Agent is the cornerstone for \
               informing trading decisions.",
    tools = [search_tool, scrape_website],
    allow_delegation=True,
    verbose=True
)

In [4]:
trading_strategy_analyst = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial markets and quantitative analysis, this agent \
               devises and refines trading strategies. It evaluates the performance of different approaches to determine \
               the most profitable and risk-averse options.",
    tools=[search_tool, scrape_website],
    allow_delegation=True,
    verbose=True
)

In [5]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, and logistical details of potential trades. By evaluating \
               these factors, it provides well-founded suggestions for when and how trades should be executed to maximize \
               efficiency and adherence to strategy.",
    tools=[search_tool, scrape_website],
    allow_delegation=True,
    verbose=True
)

In [6]:
risk_management_agent = Agent(
    role = "Risk Advisor",
    goal = "Evaluate and provide insights on the risks associated with potential trading activities.",
    backstory = "Armed with a deep understanding of risk assessment models and market dynamics, this agent scrutinizes the potential \
                 risks of proposed trades. It offers a detailed analysis of risk exposure and suggests safeguards to ensure that \
                 trading activities align with the firm’s risk tolerance.",
    tools = [search_tool, scrape_website],
    allow_delegation=True,
    verbose=True
)

## Creating Agent

In [7]:
data_analysis_task = Task(
    description="Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements.",
    expected_output="Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}.",
    agent=data_analyst_agent
)

In [8]:
trading_strategy_task = Task(
    description="Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference}).",
    expected_output= "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance.",
    agent=trading_strategy_analyst
)

In [9]:
execution_task = Task(
    description="Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing.",
    expected_output="Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}.",
    agent=execution_agent
)

In [10]:
risk_management_task = Task(
    description="Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies.",
    expected_output="A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}.",
    agent=risk_management_agent
)

## Creating Crew

In [11]:
from langchain_openai import ChatOpenAI

In [12]:
manager_llm_chatbot = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [13]:
financial_trading_crew = Crew(
    agents = [data_analyst_agent, trading_strategy_analyst, execution_agent, risk_management_agent],
    tasks = [data_analysis_task, trading_strategy_task, execution_task, risk_management_task],
    manager_llm = manager_llm_chatbot,
    process = Process.hierarchical,
    verbose=True
)

In [14]:
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [15]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


# Agent: Crew Manager
## Thought: I need to gather relevant market data and insights for AAPL. This will require me to search for current market trends, statistical models, and machine learning approaches used to predict stock movements.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"AAPL stock market trends analysis statistical modeling machine learning predictions 2023\"}"
## Tool Output: 
{'searchParameters': {'q': 'AAPL stock market trends analysis statistical modeling machine learning predictions 2023', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Apple Inc. Stock Market Analysis and Prediction Using Machine ...', 'link': 'https://www.linkedin.com/pulse/apple-inc-stock-market-analysis-prediction-using-machine-olesin

In [17]:
from IPython.display import Markdown
Markdown(result.raw)

To evaluate the risks associated with the proposed trading strategies for AAPL, it is important to consider both the macroeconomic environment and the specific risks identified from the recent analyses of the company. Here’s a detailed breakdown of potential risks and suggested mitigation strategies:

1. **Market Volatility Risks**:
   - AAPL stock is known for experiencing significant price volatility. The stock could react adversely to market conditions, economic data releases, or changes in Federal Reserve policies. 
   - **Mitigation Strategy**: Implement position sizing to control exposure and use options for hedging. A stop-loss order may also be beneficial.

2. **Regulatory and Legal Risks**:
   - AAPL operates in a highly regulated environment, facing risks related to compliance with U.S. and international laws, especially those regarding data privacy and consumer protection.
   - **Mitigation Strategy**: Continually review compliance with current regulations and stay informed on impending regulatory changes. Diversify investments to spread potential legal risks across different sectors.

3. **Technological Risks**:
   - As AAPL relies on innovations in technology, any disruptions in technology development or cybersecurity breaches could significantly impact operations.
   - **Mitigation Strategy**: Develop robust cybersecurity measures and consider insurance policies to cover potential breaches. Additionally, invest in R&D to stay ahead of technology trends.

4. **Supply Chain Disruptions**:
   - AAPL's manufacturing heavily relies on outsourced partners primarily located in regions prone to natural disasters, political unrest, or disruptions.
   - **Mitigation Strategy**: Establish multiple suppliers and geographical diversification to reduce dependence on a single source. Additionally, maintain safety stock levels where feasible.

5. **Financial Risks**:
   - Fluctuations in exchange rates, changes in interest rates, and credit risks within the investment portfolio could all affect profitability.
   - **Mitigation Strategy**: Use currency hedging strategies, regularly review financial exposure, and consider lowering debt levels to decrease financial leverage.

6. **Competitor Risks**:
   - AAPL operates in a competitive landscape with pressures on pricing and market share from both established and emerging companies.
   - **Mitigation Strategy**: Focus on brand loyalty and differentiate through unique product offerings. Monitor competitor strategies to adapt quickly to market changes.

7. **Economic and Political Risks**:
   - Political events, trade restrictions, and the global economic environment can significantly impact AAPL's operations and sales.
   - **Mitigation Strategy**: Stay flexible in operations and maintain a diverse product portfolio to adapt to changing political landscapes.

8. **Reputational Risks**:
   - Given the high-profile nature of AAPL, any negative publicity can harm brand perception.
   - **Mitigation Strategy**: Proactively manage public relations and maintain transparency regarding business practices.

In conclusion, while the proposed trading strategies leveraging machine learning and analysis provide a systematic approach to AAPL trading, it is crucial to implement these mitigation strategies to address the identified risks. Continuously monitor both internal and external factors that can influence AAPL's performance to make timely adjustments to trading strategies as needed.
```